In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [2]:
from settings  import *
from preprocessing import *


In [3]:
def process(subject):
    """
    a function to get preprocessed eeg signal for a single subject
    """
    # Bandpass filtring 
    raw = get_raw(subject)
    ica = get_ica(subject)
    
    return reconstuct_signal(raw, ica)

In [4]:
X = process("01")
X_train = X[:int(X.shape[1]*0.3)]

Opening raw data file /home/brain/openmiir/raw_data/P01-raw.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 64)  idle
    Range : 0 ... 2478165 =      0.000 ...  4840.166 secs
Ready.
Reading 0 ... 2478165  =      0.000 ...  4840.166 secs...
Reading /home/brain/openmiir/eeg/preprocessing/ica/P01-100p_64c-ica.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 64)  idle
Now restoring ICA solution ...
Ready.
Transforming to ICA space (61 components)
Zeroing out 4 ICA components


In [5]:
import torch
from torch.nn import *
from pylab import *
from torch import optim
from torch.utils import data
from torch import nn
from torch.nn.functional import soft_margin_loss


class StagerNet(Module):
    """
     StagerNet implementation.    
    """
    def __init__(self, num_classes, num_channels, temp_lenght ):
        super().__init__()      
        # create conversion layer
        self.relu = ReLU()
        self.spatial_conv = Conv2d(1, num_channels, (num_channels,1), stride= (1,1))
        self.temp_conv1 =  Conv2d(1, 16, (1,51), stride= (1,1), padding= (0,(51-1)//2))#(51-1)//2 to insure same padding
        self.batch_norm1 = BatchNorm2d(16)
        self.temp_conv2 =  Conv2d(16, 16, (1,51), stride= (1,1), padding= (0,(51-1)//2))#(51-1)//2 to insure same padding
        self.batch_norm2 = BatchNorm2d(16)
        self.maxPool = MaxPool2d((1, 13), stride=(1, 13))
        self.flatten = Flatten()
        self.dropout = Dropout(p = 0.5)
        self.linear_class = Linear(num_channels*(temp_lenght//(13*13))*16,num_classes )        
    def forward(self, inputs):
      x = self.spatial_conv(inputs)
      x = x.permute(0,2,1,3)
      x = self.relu(self.temp_conv1(x))
      # a relu activation is used before batch_norm, is it the case in the original implementation ?
      x = self.batch_norm1(x) 
      x = self.relu(x)      
      x = self.maxPool(x)
      x = self.relu(self.temp_conv2(x)) 
      x = self.batch_norm2 (x)
      x = self.relu(x)
      x = self.maxPool(x)
      x = self.dropout(self.flatten(x))
      x = self.linear_class(x)
      return x


class ShallowNet(Module):
    """
     ShallowNet implementation.    
    """
    def __init__(self, num_classes, num_channels , temp_lenght):
        super().__init__()      
        # create conversion layer
        self.eps = 1e-45
        self.relu = ReLU()
        
        self.temp_conv1 =  Conv2d(1, 40, (1,25), stride= (1,1))
        self.batch_norm2 = BatchNorm2d(40)
        self.spatial_conv = Conv2d(40, 40, (num_channels,1), stride= (1,1))
        self.meanPool = AvgPool2d((1, 75), stride=(1, 15))
        self.flatten = Flatten()
        self.dropout = Dropout(p = 0.5)
        self.num_features = (((temp_lenght-25+1)-75)//15+1)*40
        self.linear_class = Linear( self.num_features,num_classes )  

    def forward(self, inputs):
      x = self.temp_conv1(inputs)
      x = self.batch_norm2(x)
      x = self.spatial_conv(x)
      x = torch.pow(x, 2) # squaring non-linearity

      x = self.meanPool(x)
      x = self.flatten(x)
      x = torch.log(x+self.eps) #log  non-linearity 
      
      x = self.dropout(self.flatten(x))
      x = self.linear_class(x)
      return x

In [6]:
from pylab import *
import torch 
from torch.nn import *

class WeightedSampler(torch.utils.data.sampler.Sampler):
    r"""Sample des windows randomly
    Arguments:
    ---------
        dataset (Dataset): dataset to sample from
        size (int): The total number of sequences to sample
    """

    def __init__(self,dataset, batch_size,size,  weights):
    
        
        self.batch_size = batch_size
        self.size = size
        self.dataset = dataset
        self.serie_len = len(self.dataset)
        
        self.weights = torch.DoubleTensor(weights)
        
    def __iter__(self):
        num_batches = self.size// self.batch_size
        while num_batches > 0:
            #print()
            sampled = 0
            while sampled < self.batch_size:
                target  = 2*torch.multinomial(
            self.weights, 1, replacement=True) -1
                t = choice(arange(0, self.serie_len-self.dataset.temp_len, 1))
                t_ = self.dataset.get_pos(t) if target>0 else self.dataset.get_neg(t)
                sampled += 1
                yield (t, t_, target)
            
            num_batches -=1

    def __len__(self):
        return len(self.train_list)   


class Abstract_Dataset(torch.utils.data.Dataset):
    '''
    Classe dataset  pour les differents sampling
    '''
    def __init__(self, time_series, temp_len , n_features):

        self.time_series = time_series
        self.temp_len = temp_len
        self.n_features = n_features
    def get_windows(self,index):
        '''
        a method to load  a sequence 
        '''
        raise NotImplementedError
    def get_pos(self, t_anchor):
        '''
        a method to get positive samples
        '''
        raise NotImplementedError
    def get_neg(self, t_anchor):
        '''
       a method to get negative samples
        '''
        raise NotImplementedError
    def get_targets(self, index):
        '''
        a method to get labels
        '''
        raise NotImplementedError
    def __getitem__(self, index):
        windows = self.get_windows(index)
        target = self.get_targets(index)
        return windows, target
    def __len__(self): return self.time_series.shape[1]

class RP_Dataset(Abstract_Dataset):
    
    def __init__(self, time_series, sampling_params, temp_len , n_features ):
      super().__init__(time_series, temp_len = temp_len, n_features = n_features)
      self.pos , self.neg = sampling_params
    def get_windows(self,index):
        '''
        a method to get sampled windows
        '''
        (t, t_ , _) = index
        anchor_wind = self.time_series[:,t:t+self.temp_len]
        neg_wind = self.time_series[:,t_:t_+self.temp_len] # could be negative or positive
        return (anchor_wind, neg_wind)
    
    def get_targets(self, index):
        return index[-1]
    def get_pos(self, t_anchor):

      start = min(0,t_anchor-self.pos ) 
      end = min(self.__len__()-self.temp_len,t_anchor+self.pos ) # to get a sequence of lenght self.temp_lenght
      t_ = choice(arange(start,end, 1)) 
      return t_
    def get_neg(self, t_anchor):
      
      left_idx = arange(0, max(0, t_anchor - self.neg), 1)
      right_idx =arange(min(self.__len__()-self.temp_len, t_anchor + self.neg),self.__len__()-self.temp_len ,1)
      t_ = choice(hstack([left_idx, right_idx]))
      return t_

def collate(batch):
    
    anchors = torch.stack([torch.from_numpy(item[0][0]) for item in batch])
    try:
        sampled = torch.stack([torch.from_numpy(item[0][1]) for item in batch])
    except:
        print(batch)
    targets = torch.stack([item[1] for item in batch])
    
    return (anchors, sampled), targets

In [ ]:
T

In [14]:
train_dataset =  RP_Dataset(X_train, sampling_params = (600, 1000), temp_len = T ,
                            n_features = C )
train_sampler = WeightedSampler(train_dataset, batch_size = 30 ,size = 1000,  
                          weights = [0.5, 0.5])
test_sampler = WeightedSampler(test_dataset, batch_size = 30 ,size = 300,  
                          weights = [0.5, 0.5])
samplers = {"train" : train_sampler, "val": test_sampler}
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, num_workers=0,sampler = samplers["train"], collate_fn=collate)

for epoch in range(10):
    for (anchors, sampled), targets in train_loader: 
      anchors = anchors
    print(epoch)

NameError: name 'T' is not defined

In [193]:
t_anchor = 991043
print(t_anchor - test_dataset.neg)
left_idx = arange(0, max(0, t_anchor - test_dataset.neg), 1)
right_idx =arange(min(test_dataset.__len__()-test_dataset.temp_len, t_anchor + test_dataset.neg),test_dataset.__len__()-test_dataset.temp_len ,1)
t_ = choice(hstack([left_idx, right_idx]))



990043


In [8]:
rp_dataset = RP_Dataset(X, sampling_params = (600, 1000), temp_len = 200 , n_features = 69 )
sampler = WeightedSampler(rp_dataset, batch_size = 5 ,size = 10,  weights = [0.5, 0.5])
loader = torch.utils.data.DataLoader(rp_dataset, batch_size=5,  sampler=sampler,
           batch_sampler=None, num_workers=0, collate_fn=collate)

for (anchors, sampled), targets in loader: 
  print(sampled.shape)
  print(targets)

torch.Size([5, 69, 200])
tensor([[-1],
        [ 1],
        [ 1],
        [-1],
        [-1]])
torch.Size([5, 69, 200])
tensor([[-1],
        [ 1],
        [ 1],
        [-1],
        [ 1]])


In [7]:
EEG_FeatureExtractor = StagerNet(num_classes = 10, num_channels = 69, temp_lenght = 200 )
from pylab import *

inputs = sampled.unsqueeze(dim = 1)
EEG_FeatureExtractor.to(float)
y = EEG_FeatureExtractor(inputs)
y.shape

NameError: name 'sampled' is not defined

In [18]:
DEVICE

device(type='cuda')

In [9]:
from torch.nn.functional import soft_margin_loss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class Relative_Positioning(nn.Module):
  def __init__(self, EEG_FeatureExtractor, C, T, embedding_dim=100):
    super().__init__()
    self.feature_extractor = EEG_FeatureExtractor(num_classes =embedding_dim , num_channels = C , temp_lenght = T).to(float).to(device)
    #self.feature_extractor.float()
    self.linear = nn.Linear(embedding_dim, 1)
    self.loss_fn = nn.SoftMarginLoss()

  def forward(self, x):
    first_samples = x[0].unsqueeze(dim=1)
    second_samples = x[1].unsqueeze(dim=1)

    h_first = self.feature_extractor(first_samples)
    h_second = self.feature_extractor(second_samples)

    h_combined = torch.abs(h_first - h_second)

    out = self.linear(h_combined)
    return out

In [10]:
ssl_model = Relative_Positioning(StagerNet,C = 69, T = 200 )
ssl_model.to(float).to(device)

Relative_Positioning(
  (feature_extractor): StagerNet(
    (relu): ReLU()
    (spatial_conv): Conv2d(1, 69, kernel_size=(69, 1), stride=(1, 1))
    (temp_conv1): Conv2d(1, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25))
    (batch_norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (temp_conv2): Conv2d(16, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25))
    (batch_norm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (maxPool): MaxPool2d(kernel_size=(1, 13), stride=(1, 13), padding=0, dilation=1, ceil_mode=False)
    (flatten): Flatten()
    (dropout): Dropout(p=0.5, inplace=False)
    (linear_class): Linear(in_features=1104, out_features=100, bias=True)
  )
  (linear): Linear(in_features=100, out_features=1, bias=True)
  (loss_fn): SoftMarginLoss()
)

In [21]:
ssl_model(x = (anchors.to(device), sampled.to(device)))

tensor([[-0.0201],
        [ 1.6412],
        [ 0.2893],
        [ 1.0355],
        [ 1.6782]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddmmBackward>)

In [22]:
def _eval_loss(model, data_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for (anchors, sampled), y in data_loader:
            x = (anchors.to(device).to(float).contiguous(), sampled.to(device).to(float).contiguous())
            y = y.to(device).to(float).contiguous()
            loss = rp_loss(model, x, y)
            total_loss += loss * x.shape[0]
        avg_loss = total_loss / len(data_loader.dataset)
    return avg_loss.item()

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def rp_loss(model, x, y):
	out = model(x)
	return soft_margin_loss(out, y)
 
def _train(model, train_loader, optimizer, epoch):
	model.train()
	
	train_losses = []
	for (anchors, sampled), y in train_loader:
		x = (anchors.to(device).to(float).contiguous(), sampled.to(device).to(float).contiguous())
		y = y.to(device).to(float).contiguous()
		loss = rp_loss(model, x, y)
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
		train_losses.append(loss.item())
	return train_losses

def _eval_loss(model, data_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for (anchors, sampled), y in data_loader:
            x = (anchors.to(device).to(float).contiguous(), sampled.to(device).to(float).contiguous())
            y = y.to(device).to(float).contiguous()
            loss = rp_loss(model, x, y)
            total_loss += loss * x[0].shape[0]
        avg_loss = total_loss / len(data_loader.dataset)
    return avg_loss.item()

	

saved_models_dir = "saved_models"
def _train_epochs(model, train_loader, test_loader, train_args):
	epochs, lr = train_args['epochs'], train_args['lr']
	optimizer = optim.Adam(model.parameters(), lr=lr)
	if not os.path.exists(saved_models_dir):
		os.makedirs(saved_models_dir)
	
	train_losses = []
	test_losses = [_eval_loss(model, test_loader)]
	for epoch in range(1, epochs+1):
		model.train()
		train_losses.extend(_train(model, train_loader, optimizer, epoch))
		test_loss = _eval_loss(model, test_loader)
		test_losses.append(test_loss)
		print(f'Epoch {epoch}, Test loss {test_loss:.4f}')
		
		# save model every 10 epochs
		if epoch % 2 == 0:
			torch.save(model.state_dict(), os.path.join(ROOT, 'saved_models', 'ssl_model_epoch{}.pt'.format(epoch)))
	torch.save(model.state_dict(), os.path.join(root, 'saved_models', 'ssl_model.pt'))
	return train_losses, test_losses


def train_ssl(model, train_dataset, test_dataset,sampler, n_epochs=20, lr=1e-3, batch_size=256, load_last_saved_model=False, num_workers=8):
	C = train_dataset.__getitem__((0, 0,1))[0][0].shape[0] # num channels
	T = train_dataset.__getitem__((0, 0,1))[0][0].shape[1] # num timepoints
	
	if load_last_saved_model:
		model.load_state_dict(torch.load(os.path.join(root, 'saved_models', 'ssl_model.pt')))

	if torch.cuda.device_count() > 1:
		model = nn.DataParallel(model)
        
	model.to(device)
    
   

	train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers,sampler = sampler["train"], collate_fn=collate)
	test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers,sampler = sampler["val"], collate_fn=collate)

	new_train_losses, new_test_losses = _train_epochs(model, train_loader, test_loader, 
																				 dict(epochs=n_epochs, lr=lr))

	if load_last_saved_model:
		train_losses, test_losses = load_losses(saved_models_dir, 'ssl')
	else:
		train_losses = []
		test_losses = []
	
	train_losses.extend(new_train_losses)
	test_losses.extend(new_test_losses)

	save_losses(train_losses, test_losses, saved_models_dir, 'ssl')

	return train_losses, test_losses, model


In [14]:
parameters = {
    "n_features" : 69,
    "temp_lenght": 200,
    "sampling_params" : {"pos": 600, "neg": 1000},
    "batch_size": 30,
    "train_size": 1000,
    "test_size": 300,
    "sampling_weights": [0.5, 0.5],
    "n_epochs" : 15,
    "lr": 1e-3
    }

In [13]:
#def get_neg(self, t_anchor):
      
    #left_idx = arange(0, max(0, t_anchor - self.neg), 1)
    #right_idx =arange(min(self.__len__()-self.temp_len, t_anchor + self.neg),self.__len__()-self.temp_len ,1)
    #try :
       # t_ = choice(hstack([left_idx, right_idx]))
    #except:
        #print("t_", t_anchor)
                
    #return t_
#RP_Dataset.get_neg = get_neg
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
root = ROOT

C = 69
T = 200
split = int(X.shape[1]*0.6)
X_train = X[:, :split]
X_test = X[:, split:]
ssl_model = Relative_Positioning(StagerNet,C , T )
ssl_model.to(float)
train_dataset =  RP_Dataset(X_train, sampling_params = (600, 1000), temp_len = T ,
                            n_features = C )
test_dataset =  RP_Dataset(X_test, sampling_params = (600, 1000), temp_len = T ,
                            n_features = C )
train_sampler = WeightedSampler(train_dataset, batch_size = 30 ,size = 1000,  
                          weights = [0.5, 0.5])
test_sampler = WeightedSampler(test_dataset, batch_size = 30 ,size = 300,  
                          weights = [0.5, 0.5])
samplers = {"train" : train_sampler, "val": test_sampler}

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10, num_workers=0,sampler = samplers["val"], collate_fn=collate)

train_losses, test_losses, model = train_ssl(ssl_model, train_dataset, test_dataset,
                                             samplers,n_epochs=15, lr=1e-3,batch_size=30, 
                                             load_last_saved_model=False, num_workers= 0)




Epoch 1, Test loss 0.0003
Epoch 2, Test loss 0.0002
Epoch 3, Test loss 0.0008
Epoch 4, Test loss 0.0004
Epoch 5, Test loss 0.0002
Epoch 6, Test loss 0.0003
Epoch 7, Test loss 0.0003
Epoch 8, Test loss 0.0002
Epoch 9, Test loss 0.0003
Epoch 10, Test loss 0.0002
Epoch 11, Test loss 0.0002
Epoch 12, Test loss 0.0002
Epoch 13, Test loss 0.0002
Epoch 14, Test loss 0.0002
Epoch 15, Test loss 0.0002


NameError: name 'save_losses' is not defined

In [21]:
!python3 main.py --epochs 5


Training parameters:
- subject: 01
- C : 69
- T : 1000
- epochs : 5
- batch size : 30
- lr : 0.001
- n_train : 3000
- n_test : 300
- pos : 600
- neg : 600
Opening raw data file /home/brain/openmiir/raw_data/P01-raw.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 64)  idle
    Range : 0 ... 2478165 =      0.000 ...  4840.166 secs
Ready.
Reading 0 ... 2478165  =      0.000 ...  4840.166 secs...
Reading /home/brain/openmiir/eeg/preprocessing/ica/P01-100p_64c-ica.fif ...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 64)  idle
Now restoring ICA solution ...
Ready.
Transforming to ICA space (61 components)
Zeroing out 4 ICA components
Epoch 1, Test loss 0.0004
Epoch 2, Test loss 0.0007
Epoch 3, Test loss 0.0003
Epoch 4, Test loss 0.0005
Epoch 5, Test loss 0.0003
